In [ ]:
#測試抓愛食記網站
import requests
from bs4 import BeautifulSoup
from datetime import date,datetime
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import csv
import pandas as pd
import json

In [ ]:
def get_comment(url):
    browser = webdriver.Chrome(executable_path='/usr/bin/chromedriver')
    browser.get(url)
    browser.implicitly_wait(1)
    time.sleep(1)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    while len(soup.select('.user-checkin > .content > .btn-more-checkin-wrapper')) > 0:
        browser.find_element_by_xpath("//button[@class='jss76 jss50 jss52 jss55 btn-more-checkin']").click()
        time.sleep(1)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
     
    comment_all = []
    comment_member = {}
    for i in range(len(soup.select('.checkin-list-outer > .comment-block-outer'))):   
        #comment_member[i] = soup.select('.checkin-list-outer > .comment-block-outer')[i].text
        #comment_all.append(comment_member[i])
        #soup.select('.checkin-list-outer > .comment-block-outer')[i].text
        c = soup.select('.checkin-list-outer > .comment-block-outer > .comment-block > .comment-content-outer')[i].text
        print(c)
        #print(c[0].text)
        #print(c[1].text)
        #print(c[2].text)
        #print(c[3].text)
    #print(comment_member)
        
        
    time.sleep(5)
    browser.close() 
    #return 

In [ ]:
def pageDetail(detail_url, resturant_series_num):
    #print("url:")
    #print( detail_url)
    save_data = {}
    res = requests.get(detail_url)
    soup =  BeautifulSoup(res.text)
    
    #test tool
    #tag = input("請輸入定位元素，class前面加上.，id前面加上# ")  
    #for drink in soup.select('{}'.format(tag)):
        #print(drink.get_text())
    save_data['resturant_series_num'] = resturant_series_num
    
    name =  soup.select('.title-outer > .title')[0].text
    #print(name)
    save_data['name'] = name
    #print(save_data)
    
    star = soup.select('.jsx-1207467136')[0].text
    #print(star)
    save_data['star'] = star
    #print(save_data)
    
    comment_num = soup.select('.count')[0].text
    comment_num = int(comment_num[:-3])
    #print(comment_num)
    save_data['comment_num'] = comment_num
    #print(save_data) 
    
    data = soup.select('.info-outer > .info > div > span')
    address = data[1].text
    #print(address)
    save_data['address'] = address
    #print(save_data)
    
    phone = data[3].text
    #print(phone)
    save_data['phone'] = phone
    #print(save_data)
    
    avg_price_check = data[4].text
    #print(avg_price0)
    if avg_price_check == "均消價位|":
        avg_price = data[5].text
        avg_price = avg_price[3:]
    else:
        avg_price = 0
    #print(avg_price)
    save_data['avg_price'] = avg_price
    #print(save_data)
    
    
    #get_comment(detail_url)

    print(save_data)
    print('-------------------------')
    return save_data
    
    

In [ ]:
def deal_with_resturant_series_num(num):
    RSN = 'RSN_0'
    length = len(str(num))
    if length == 1:
        RSN = RSN + '0000' + str(num)
    elif length == 2:
        RSN = RSN + '000' + str(num)
    elif length == 3:
        RSN = RSN + '00' + str(num) 
    elif length == 4:
        RSN = RSN + '0' + str(num)  
    elif length == 5:
        RSN = RSN + str(num)      
    #print(RSN)
    return RSN

In [ ]:
test_RSN = deal_with_resturant_series_num(657)
print(test_RSN)

In [ ]:
domain = 'https://ifoodie.tw'
resturant_ary = []
counter = 0
resturant_series_num_counter = 0
for page in range(1,2):
    resturant_url = 'https://ifoodie.tw/explore/%E5%8F%B0%E5%8C%97%E5%B8%82/list?page={}'
    section = requests.get(resturant_url.format(page))
    soup = BeautifulSoup(section.text)
    counter = 0
    
    resturant_series_num = deal_with_resturant_series_num(resturant_series_num_counter)
    
    resturant_data = {}
    resturant_type = {}
    for resturant in soup.select('.restaurant-info'):      
        catergory = soup.select('.category-row')[counter].text
        catergory = catergory[4:]
        #print(catergory)
        if catergory != '':
            resturant_type['type'] = catergory
        else:
            resturant_type['type'] = '未定義'
            
        #print(resturant_type)
        dic = {}
        dic['link'] = domain + resturant.select('a')[0]['href']
        resturant_data = pageDetail(dic['link'] , resturant_series_num)
        resturant_data.update(resturant_type)
        resturant_ary.append(resturant_data)
        counter = counter + 1
        resturant_series_num_counter = resturant_series_num_counter + 1

In [ ]:
for j in resturant_ary:
    print(j)
    print('------------------------------------------------------------------------------------------------------------------------------------------------------------')

In [ ]:
sub = pd.DataFrame(resturant_ary1)

In [ ]:
sub

In [ ]:
sub.to_csv("test.csv", index=False)

In [ ]:
sub.to_json(r'test.json')